In [1]:
## Getting content from tavily APIs

user_query = "What are the effects of parental involvement on academic performance?"
#user_query = "How can a father and mother influence a child's performance and grades in school?"

from tavily import TavilyClient
import shutil
import os

tavily = TavilyClient(api_key=os.getenv('TAVILY_API_KEY'))

response = tavily.search(query=user_query, 
                         search_depth="advanced", 
                         include_raw_content=True,
                         max_results=5)

HTTPError: 422 Client Error: Unprocessable Entity for url: https://api.tavily.com/search

In [21]:
if os.path.exists('tavily_data'):
    shutil.rmtree('tavily_data')
    os.makedirs('tavily_data')
else:
    os.makedirs('tavily_data')

for count, result in enumerate(response['results']):
    f = open("tavily_data/demofile{}.txt".format(count), "a", encoding="utf-8")
    f.truncate(0)
    f.write(result['raw_content'])
    f.close()

In [22]:
## querying AI model

from pathlib import Path
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
)
from llama_index.llms.openai import OpenAI
from typing import List, Optional
from llama_index.llms.huggingface import (
    HuggingFaceInferenceAPI,
    HuggingFaceLLM,
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from pydantic import BaseModel
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.schema import IndexNode
from llama_index.core.embeddings import resolve_embed_model
from llama_index.core.response.notebook_utils import display_source_node
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import MetadataReplacementPostProcessor

In [23]:
from llama_index.core import Settings
Settings.llm = OpenAI(model="gpt-4", api_key=os.getenv("OPENAI_API_KEY"))

In [9]:
documents = SimpleDirectoryReader("tavily_data").load_data()
index = VectorStoreIndex.from_documents(documents)

In [24]:
# HF_TOKEN: Optional[str] = os.getenv("HUGGING_FACE_TOKEN")
gpt4 = OpenAI(model="gpt-4", api_key=os.environ['OPENAI_API_KEY'])
query = user_query

In [25]:
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)
text_splitter = SentenceSplitter()
embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/all-mpnet-base-v2", max_length=512
)
Settings.embed_model = embed_model
Settings.text_splitter = text_splitter

nodes = node_parser.get_nodes_from_documents(documents)
base_nodes = text_splitter.get_nodes_from_documents(documents)

sentence_index = VectorStoreIndex(nodes)
base_index = VectorStoreIndex(base_nodes)


In [26]:
query_engine = sentence_index.as_query_engine(
    similarity_top_k=2,
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
)
window_response = query_engine.query(query)

for window in window_response.source_nodes:
    print("Window: ")
    print(window.node.metadata["window"])
    print("Original Sentence: ")
    print(window.node.metadata["original_text"])
    print()

print(window_response)


Window: 
Therefore, it is important to examine factors that contribute to early academic success and that are amenable to change.
 Researchers have reported that parent-child interactions, specifically stimulating and responsive parenting practices, are important influences on a child's academic development (Christian, Morrison, & Bryant, 1998; Committee on Early Childhood Pedagogy, 2000).  By examining specific parenting practices that are amenable to change, such as parent involvement, and the mechanisms by which these practices influence academic performance, programs may be developed to increase a child's academic performance.  While parent involvement has been found to be related to increased academic performance, the specific mechanisms through which parent involvement exerts its influence on a child's academic performance are not yet fully understood (Hill & Craft, 2003).  Understanding these mechanisms would inform further research and policy initiatives and may lead to the dev

In [8]:
base_retriever = index.as_retriever(similarity_top_k=5)
retrievals = base_retriever.retrieve(query)
for n in retrievals:
    if (not n.text.__contains__("http")):
        display_source_node(n, source_length=1500)
query_engine_base = RetrieverQueryEngine.from_args(base_retriever, llm=gpt4)
response = query_engine_base.query(query)
print(str(response))

**Node ID:** 42d14a4e-b3af-4fa9-80af-dfb165064692<br>**Similarity:** 0.8463510220273155<br>**Text:** Parents who communicate with their children, visit museums, or record daily activities can cultivate children’s information literacy, improve math and reading scores, and directly stimulate cognitive development (Sibley and Dearing, 2014; Choe, 2020). Moreover, family socioeconomic status (SES), related developmental resources (including parental support, expectations, and reading resources), and students’ individual reading motivation (including reading engagement and reading confidence) also affect adolescents’ learning outcomes (including academic achievement, school grades, and reading competence) (Mudrak et al., 2020).
With the increase in communication time between adolescent students and their peers, they gradually break away from their families to participate in peer interaction; peers have become another major field affecting their development (Criss et al., 2016). The interpersonal relationship established by teenagers in the school field plays an essential role in their behavior development, cognitive ability, and academic performance, especially adolescents’ academic engagement or learning performance is influenced by friendship quality (Sebanc et al., 2016), friends’ gain and loss (Lessard and Juvonen, 2018), and peer personality (Golsteyn et al., 2021) factors in peer groups. Reviewing the research on the influence of family and important others on students’ achievement, there are two main points of view: On the one hand, according to American psychologist Ha...<br>

**Node ID:** ab61e1b0-11c1-464e-baf4-d9ac1c45f536<br>**Similarity:** 0.8429619168263194<br>**Text:** Second, the influence of peers and family on adolescents is a “complementary” view (Fukuoka and Hashimoto, 1997), that is, peer interaction transmits the effect of family environment on adolescents’ academic performance, compared with childhood, at the adolescent stage, parent education and parent-child interaction no longer meet their needs, but gradually extend to seeking support or help among peers, and peer interaction and family environment together influence adolescents’ growth. The family environment (parenting style, behavior supervision, and emotional intervention) plays a decisive role in the quality of peer interaction among adolescents; for example, parents supervise their children’s home time, places to go out, friend interactions, and homework completion, which would increase children’s exposure to peers with positive learning behaviors (Deutsch et al., 2012; DeAnna, 2016), which indicates that parent–child communication and parental educational involvement influence children’s interpersonal interactions. At the same time, the peer network structure, friend quality, and personality orientation also affect students’ academic achievement (Berthelon et al., 2019; Golsteyn et al., 2021). Based on the available research, a more superior family environment may have a positive effect on students’ academic achievement by increasing their peer interaction quality.
Moderating effect of educational expectation gap
Educational expectation is based on one’s cognitive abil...<br>

Parents can significantly influence a child's academic performance and grades in several ways. They can cultivate their child's information literacy, improve math and reading scores, and directly stimulate cognitive development by communicating with their children, visiting museums, or recording daily activities. Family socioeconomic status (SES), related developmental resources such as parental support, expectations, and reading resources, and students’ individual reading motivation also affect adolescents’ learning outcomes. Parental involvement in education, supervision of their children's activities, and emotional intervention can also play a decisive role in the quality of peer interaction among adolescents, which in turn influences students' academic achievement. Furthermore, parents' educational expectations for their children can impact their academic performance. If these expectations are moderate and align with the child's self-expectations, it can be conducive to the child's